In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import glob
import os

In [ ]:
%matplotlib inline

# 1d case

In [ ]:
!ls 1d

In [ ]:
fs_imex_all = []
fs_ce_all = []
kappas = [1e-3, 1e-4, 1e-5]
for n in range(3, 3+len(kappas)):
    files_imex = sorted(glob.glob(f'1d/data_imex_*_kappa1em{n}.hdf5'), key=os.path.getmtime)
    fs_imex = []
    for f in files_imex:
        fs_imex.append(h5py.File(f, 'r'))
    fs_imex_all.append(fs_imex)
    files_ce = sorted(glob.glob(f'1d/data_ce_*_kappa1em{n}.hdf5'), key=os.path.getmtime)
    fs_ce = []
    for f in files_ce:
        fs_ce.append(h5py.File(f, 'r'))
    fs_ce_all.append(fs_ce)

In [ ]:
print(len(fs_imex_all[0]), len(fs_ce_all[0]))
print(len(fs_imex_all[1]), len(fs_ce_all[1]))
print(len(fs_imex_all[2]), len(fs_ce_all[2]))

In [ ]:
for n, kappa in enumerate(kappas):
    fig, axes = plt.subplots(len(fs_imex_all[n]), 2, figsize=(8, 20))
    for i in range(len(fs_imex_all[n])):
        axes[i, 0].plot(fs_imex_all[n][i]['Domain/x'], fs_imex_all[n][i]['Primitive/T'][:], label='IMEX')
        axes[i, 0].plot(fs_ce_all[n][i]['Domain/x'], fs_ce_all[n][i]['Primitive/T'][:], label='CE')
        axes[i, 0].legend()
        axes[i, 0].set_title(f"nx = {fs_imex_all[n][i]['Domain'].attrs['nx'][0]}")
        axes[i, 0].set_ylabel(r"$T$")
        axes[i, 1].plot(fs_imex_all[n][i]['Domain/x'], fs_imex_all[n][i]['Primitive/T'][:]-fs_ce_all[n][i]['Primitive/T'][:], label='IMEX')
        axes[i, 1].set_title("IMEX - CE")
    axes[-1, 0].set_xlabel(r"$x$")
    axes[-1, 1].set_xlabel(r"$x$")
    fig.suptitle(fr"$\kappa=10^{{-{3+n}}}$")
    fig.tight_layout()
    plt.show()

Needs self-convergence tests here as well.

This is not correct and needs me to check the right thing.

In [ ]:
diff2_all = []
for n in range(len(fs_imex_all)):
    diff2 = []
    for i in range(len(fs_imex_all[n])-1):
        Tlo = fs_imex_all[n][i]['Primitive/T'][:]
        Thi = fs_imex_all[n][i+1]['Primitive/T'][:]
        Tdiff = np.repeat(Tlo, 2) - Thi
        diff2.append(np.sqrt(np.sum(Tdiff**2))/np.sqrt(fs_imex_all[n][i]['Domain'].attrs['dx']))
    diff2_all.append(diff2)
print(diff2_all)

In [ ]:
plt.plot(fs_imex_all[0][0]['Domain/x'], fs_imex_all[0][0]['Primitive/T'][:])
plt.plot(fs_imex_all[0][1]['Domain/x'], fs_imex_all[0][1]['Primitive/T'][:])